MSK MRI 자동 추출기

In [ ]:
import openpyxl
import pandas as pd
import os
import tkinter as tk
from tkinter import filedialog

In [ ]:
## 파일 경로 받아오는 GUI 호출

root = tk.Tk()
root.withdraw()
dir_path = filedialog.askopenfile(
    parent=root,initialdir=os.getcwd(),title='Please select a file',
    filetypes=(('xls files','*.xls'),('all files','*.*')))

In [ ]:
## 파일 읽어서 Data Frame으로 저장

print("load file: ", dir_path.name)
df = pd.read_excel(dir_path.name, sheet_name=None)

mri_table = pd.DataFrame()
mri_table = pd.concat(df, ignore_index=True)

In [ ]:
# 필요없는 값들 다 지우기
mri_table.drop([0, 1, 2, 3], inplace=True)
mri_table.dropna(axis=1, inplace=True)
mri_table.rename(columns=mri_table.iloc[0],inplace=True)
mri_table.drop([4], inplace=True)
mri_table.drop(["주민번호","진료일자","병실"],axis=1, inplace=True)

# str --> int 형변환
mri_table["NO"] = pd.to_numeric(mri_table["NO"])
mri_table["등록번호"] = pd.to_numeric(mri_table["등록번호"])

In [ ]:
## remove list 파일 읽어서 Data Frame으로 저장

remove_list_path = "./msk/remove_list.xlsx"
df2 = pd.read_excel(remove_list_path, sheet_name=None, engine='openpyxl')

remove_list_table = pd.DataFrame()
remove_list_table = pd.concat(df2, ignore_index=True)

In [ ]:
## remove list와 일치하는 처방 삭제하기

for i in range(len(remove_list_table)):
    mri_table = mri_table[~mri_table["처방명"].str.contains(remove_list_table["키워드"][i])]

# No 1부터 다시 매기기
mri_table["NO"] = range(1, len(mri_table)+1)

In [ ]:
## data frame을 openpyxl worksheet로 전환

from openpyxl import Workbook
from openpyxl.utils.dataframe import dataframe_to_rows

wb = Workbook()
ws = wb.active

for r in dataframe_to_rows(mri_table, index=False, header=True):
    ws.append(r)


In [ ]:
## 서식 넣기

from openpyxl.styles import Alignment, Font, Border, Side
align_center = Alignment(horizontal='center', vertical='center', wrap_text= True)
font_9 = Font(name='굴림', size=9, bold=False)
border_thin = Border(left=Side(style='thin'),right=Side(style='thin'),top=Side(style='thin'),bottom=Side(style='thin'))
col_max = ws.max_column+2

for x in range(1, ws.max_row + 1):
    for y in range(1, col_max):
        ws.cell(row=x, column=y).alignment = align_center
        ws.cell(row=x, column=y).font = font_9
        ws.cell(row=x, column=y).border = border_thin

In [ ]:
## 셀 크기 조정
for row in range(1,ws.max_row+1):
    ws.row_dimensions[row].height = 27.75

from openpyxl.utils import get_column_letter
for col in range(1,ws.max_column+2):
    if col == 5:
        ws.column_dimensions[get_column_letter(col)].width = 48
    elif col == 9:
        ws.column_dimensions[get_column_letter(col)].width = 48
    elif col == 10:
        ws.column_dimensions[get_column_letter(col)].width = 32
    else:
        ws.column_dimensions[get_column_letter(col)].width = 13.6

In [ ]:
## 최종 결과물을 엑셀 파일로 저장

wb.save("msk_result.xlsx")